# Quantitative evaluation with a workflow

It is crucial to qualitatively the performance of anomaly detectors to know their capabilities. For this, ``dtaianomaly`` offers the [``Workflow``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/workflow.html#dtaianomaly.workflow.Workflow): detect anomalies in a large set of time series using various detectors, and to measure their performance using multiple evaluation criteria. The [``Workflow``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/workflow.html#dtaianomaly.workflow.Workflow) facilitates the validation of the anomaly detectors, because you only need to define
the different components.

There are two ways to run a [``Workflow``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/workflow.html#dtaianomaly.workflow.Workflow) from Python or from a configuration file.

> You can also evaluate [custom components](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/getting_started/custom_models.html) in ``dtaianomaly`` via a [``Workflow``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/workflow.html#dtaianomaly.workflow.Workflow) in Python. However, this is not possible via a configuration file without extending the functionality of the [workflow_from_config](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/workflow.html#dtaianomaly.workflow.workflow_from_config) function!

In [1]:
from dtaianomaly.workflow import Workflow
from dtaianomaly.data import UCRLoader
from dtaianomaly.evaluation import AreaUnderROC, AreaUnderPR, Precision
from dtaianomaly.thresholding import TopN, FixedCutoff
from dtaianomaly.preprocessing import Identity, ZNormalizer, ChainedPreprocessor, MovingAverage, ExponentialMovingAverage
from dtaianomaly.anomaly_detection import LocalOutlierFactor, IsolationForest

## Run a workflow from Python

Here we will initialize the different components to evaluate in the [``Workflow``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/workflow.html#dtaianomaly.workflow.Workflow). We start by creating a list of [``LazyDataLoader``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/data.html#dtaianomaly.data.LazyDataLoader) objects. We manually selected two time series to use for evaluation, but alternatively you can use all datasets in some directory using the [``from_directory()``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/data.html#dtaianomaly.data.from_directory) method in the data module. 

In [2]:
dataloaders = [
    UCRLoader('../data/UCR-time-series-anomaly-archive/001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_52620.txt'),
    UCRLoader('../data/UCR-time-series-anomaly-archive/002_UCR_Anomaly_DISTORTED2sddb40_35000_56600_56900.txt')
]

Next, we initialize a number of [``Preprocessor``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/preprocessing.html#dtaianomaly.preprocessing.Preprocessor)s. Below, we create 4 preprocessors to analyze the effect of Z-normalization combined with smoothing. We also add the [``Identity``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/preprocessing.html#dtaianomaly.preprocessing.Identity) preprocessor, to analyze what happens if no preprocessing is done. 

In [3]:
preprocessors = [
    Identity(),
    ZNormalizer(),
    ChainedPreprocessor([MovingAverage(10), ZNormalizer()]),
    ChainedPreprocessor([ExponentialMovingAverage(0.8), ZNormalizer()])
]

We will now initialize our anomaly detectors. Each anomaly detector will be combined with each preprocessor, and applied to each time series. 

In [4]:
detectors = [LocalOutlierFactor(50), IsolationForest(50)]

Finally, we need to define the [``Metric``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/evaluation.html#dtaianomaly.evaluation.Metric)s used to evaluate the models. Both [``BinaryMetric``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/evaluation.html#dtaianomaly.evaluation.BinaryMetric) and [``ProbaMetric``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/evaluation.html#dtaianomaly.evaluation.ProbaMetric) can be provided. However, the workflow evaluates the scores obtained by the [``predict_proba``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/anomaly_detection.html#dtaianomaly.anomaly_detection.BaseDetector.predict_proba) method of the [``BaseDetector``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/anomaly_detection.html#basedetector). To evaluate a [``BinaryMetric``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/evaluation.html#dtaianomaly.evaluation.BinaryMetric), a number of thresholding strategies must be provided to convert the continuous anomaly probabilities to discrete anomaly labels. Each thresholding strategy is combined with each thresholding metric. The thresholds have no effect on the [``ProbaMetric``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/evaluation.html#dtaianomaly.evaluation.ProbaMetric)s.

> To save on computational resources, the anomaly detector is used once to detect anomalies in a time series, and the predicted anomaly scores are used to evaluate all anomaly scores. This means that there is no computational overhead on providing more metrics, besides the resources required to compute the metric. 

In [5]:
thresholds = [TopN(20), FixedCutoff(0.5)]
metrics = [Precision(), AreaUnderPR(), AreaUnderROC()]

Once all components are defined, we initialize the [``Workflow``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/workflow.html#dtaianomaly.workflow.Workflow). We also define additional parameters, such ``n_jobs``, to allow for multiple anomaly detectors to detect anomalies in parallel. Then, we can execute the workflow by calling the [``run()``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/workflow.html#dtaianomaly.workflow.Workflow.run) method, which returns a dataframe with the results. 

In [6]:
workflow = Workflow(
    dataloaders=dataloaders,
    metrics=metrics,
    thresholds=thresholds,
    preprocessors=preprocessors,
    detectors=detectors,
    n_jobs=4
)
workflow.run()

,Dataset,Detector,Preprocessor,Runtime [s],TopN_20_precision,FixedCutoff_0.5_precision,auc-pr,auc-roc
0,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,LocalOutlierFactor_50_1,identity,13.408890,0.0,0.000000,0.008296,0.415101
1,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,IsolationForest_50_1,identity,2.444013,1.0,0.104917,0.286130,0.707950
2,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,LocalOutlierFactor_50_1,z_normalizer,13.684784,0.0,0.000000,0.008296,0.415101
3,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,IsolationForest_50_1,z_normalizer,2.588817,1.0,0.129229,0.276696,0.725667
4,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,LocalOutlierFactor_50_1,moving_average_10->z_normalizer,26.102083,0.0,0.000000,0.010392,0.422901
5,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,IsolationForest_50_1,moving_average_10->z_normalizer,12.533251,1.0,0.115420,0.295997,0.725168
6,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,LocalOutlierFactor_50_1,exponential_moving_average_0.8->z_normalizer,15.494961,0.0,0.000000,0.010401,0.432582
7,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,IsolationForest_50_1,exponential_moving_average_0.8->z_normalizer,2.585086,1.0,0.103148,0.276048,0.713873
8,002_UCR_Anomaly_DISTORTED2sddb40_35000_56600_5...,LocalOutlierFactor_50_1,identity,15.219988,0.0,0.000000,0.009152,0.495514
9,002_UCR_Anomaly_DISTORTED2sddb40_35000_56600_5...,IsolationForest_50_1,identity,2.811340,0.0,0.000000,0.001953,0.103111


## Run a workflow from a configuration file

Alternatively, you can define a workflow using JSON configuration files. The file [Config.json]() illustrates how the workflow defined above can be written as a configuration file. More details regarding the syntax are provided below. Using the [workflow_from_config](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/workflow.html#dtaianomaly.workflow.workflow_from_config) method, you can pass the path to a configuration file to create the corresponding [``Workflow``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/workflow.html#dtaianomaly.workflow.Workflow), as shown in the example below. Then, you can run the [``Workflow``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/workflow.html#dtaianomaly.workflow.Workflow) via the [``run()``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/workflow.html#dtaianomaly.workflow.Workflow.run) function. 
 

In [7]:
from dtaianomaly.workflow import workflow_from_config
workflow = workflow_from_config("Config.json")
workflow.run()

,Dataset,Detector,Preprocessor,Runtime [s],TopN_20_precision,FixedCutoff_0.5_precision,auc-pr,auc-roc
0,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,LocalOutlierFactor_50_1,identity,12.910221,0.0,0.000000,0.008296,0.415101
1,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,IsolationForest_50_1,identity,2.603895,1.0,0.099959,0.294506,0.718991
2,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,LocalOutlierFactor_50_1,z_normalizer,12.711074,0.0,0.000000,0.008296,0.415101
3,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,IsolationForest_50_1,z_normalizer,1.877480,1.0,0.111563,0.299992,0.721451
4,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,LocalOutlierFactor_50_1,moving_average_10->z_normalizer,24.272107,0.0,0.000000,0.010392,0.422901
5,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,IsolationForest_50_1,moving_average_10->z_normalizer,9.505384,1.0,0.121656,0.291344,0.724132
6,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,LocalOutlierFactor_50_1,exponential_moving_average_0.8->z_normalizer,14.763831,0.0,0.000000,0.010401,0.432582
7,001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_5...,IsolationForest_50_1,exponential_moving_average_0.8->z_normalizer,2.102279,1.0,0.127128,0.297306,0.721641
8,002_UCR_Anomaly_DISTORTED2sddb40_35000_56600_5...,LocalOutlierFactor_50_1,identity,14.497014,0.0,0.000000,0.009152,0.495514
9,002_UCR_Anomaly_DISTORTED2sddb40_35000_56600_5...,IsolationForest_50_1,identity,1.886509,0.0,0.000000,0.001974,0.114592


A configuration file is build from different entries, with each entry representing a component of the [``Workflow``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/workflow.html#dtaianomaly.workflow.Workflow). These entries are build as follows:
```json
    { 'type': <name-of-component>, 'optional-param': <value-optional-parameter>}
```
The ``'type'`` equals the name of the component, for example ``'LocalOutlierFactor'`` or ``'ZNormalizer'``. This string must exactly match the object name of the component you want to add to the workflow. In addition, it is possible to define hyperparameters of the component. For example for ``'LocalOutlierFactor'``, you must define a
``'window_size'``, but can optionally also define a ``'stride'``. An error will be raised if the entry has missing obligated parameters or unknown parameters.

The configuration file itself is also a dictionary, in JSON format. The keys of this dictionary correspond to the parameters of the [``Workflow``](https://m-group-campus-brugge.pages.gitlab.kuleuven.be/dtai_public/dtaianomaly/api/workflow.html#dtaianomaly.workflow.Workflow). The corresponding values can be either a single entry (if one component is requested) or a list of entries (if multiple components are requested).

Below, we show a simplified version of the configuration in [Config.json](). 

```json
{
  "dataloaders": {
    "type": "UCRLoader",
    "path":"../data/UCR-time-series-anomaly-archive/001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_52620.txt"
  },
  "metrics": [{"type": "Precision"}, {"type": "AreaUnderPR"}],
  "thresholds": {"type": "FixedCutoff", "cutoff": 0.5},
  "preprocessors": {"type": "ZNormalizer"},
  "detectors": [
    {"type": "LocalOutlierFactor", "window_size": 50},
    {"type": "IsolationForest", "window_size": 50}
  ],
  "n_jobs": 4
}
```